In [ ]:
from google.colab import drive

# Mount Google Drive (authentication required first)
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
webmd_base = 'https://www.webmd.com'

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from  more_itertools import unique_everseen

# url ='https://www.webmd.com/drugs/2/index'
# a = requests.get(url)
# soup = BeautifulSoup(a.content)
#print(soup)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
generic_drugs = []
alpha_drug_name_tags = []
alphabet_list = list(map(chr, range(ord('a'), ord('z') + 1)))
#print('https://www.webmd.com' + '/drugs/2/alpha/' + str(alphabet_list[0]))

for ap in alphabet_list:
    url2 = 'https://www.webmd.com' + '/drugs/2/alpha/' + ap
    r = requests.get(url2, headers=headers)
    soup = BeautifulSoup(r.content)
    new_tags = soup.find_all('a', class_='alpha-drug-name')
    alpha_drug_name_tags.extend(list(new_tags))
    #print(soup)


# for tag in soup.findAll('ul',{'class':'browse-letters squares'}):
#     print(tag)
#     for t in tag.find_all('a', href=True):
#         alphabet_list.append(t.get('href'))

# for element in alphabet_list:
#     url2 = 'https://www.webmd.com' + '/drugs/2/alpha/' + str(element)
#     r = requests.get(url2)
#     soup = BeautifulSoup(r.content)
#     print(soup)

#     # for tag in soup.findAll('ul',{'class':'alpha-drugs'}):
#     #     print(tag)
#     #     # list1 = []
#     #     # for t in tag.find_all('a', href=True):
#     #     #     list1.append(t.get_text())
#     #     #     list1.append(t.get('href'))
#     #     #     generic_drugs.append(list1)
#     #     #     list1 = []

#     alpha_drug_name_tags = soup.find_all('a', class_='alpha-drug-name')

#     # Extract and print the text from each anchor tag
#     for tag in alpha_drug_name_tags:
#         drug_name = tag.text.strip()
#         print(drug_name)


# print(generic_drugs)


In [ ]:
#alpha_drug_name_tags = soup.find_all('a', class_='alpha-drug-name')

for tag in alpha_drug_name_tags:
    drug_name = tag.text.strip()
    print(drug_name)

a and d diaper rash cream
a thru z advanced formula
a thru z select women's tablet
a-c-e-zinc ox-selen aa-copper tablet
a-carn 100 mg-a-cyst 37.5 mg-ala 25 mg-q10 mg-p.serine-broc capsule
aa 2%-d10w-low calcium-heparin solution, intravenous
aa 2.5%-d10w-calcium-heparin solution, intravenous
aa 3%-d10w-low calcium-heparin solution, intravenous
aa 3%-d5w-calcium-heparin solution, intravenous
aa 3.5%-d10w-calcium-heparin solution, intravenous
aa 4%-d10w-calcium-heparin solution, intravenous
aa-protein-selen-dex-biofl-blu capsule
b activ 400 mcg dfe capsule multivitamins
b and c topical ointment not applicable
b complex
b complex 1 (with folic acid) 0.4 mg tablet multivitamins
b complex-c-e-fa-zn-lysine tablet
b complex-e-fa-min comb#35-soy tablet
b complex-vitamin c 20 mg-5 mg-2 mg-75 mcg chewable tablet multivitamins
b complx-c-folic-zinc-copper-e tablet
b-100 complex
b-100 complex tablet, extended release
b-12 compliance kit
b-12 dots
b-12 lozenge
b-12 plus 5,000 mcg-100 mcg sublingual 

In [ ]:
print(type(alpha_drug_name_tags))

<class 'list'>


In [ ]:
def remove_html_tags(text):
    # Parse the HTML content
    soup = BeautifulSoup(text, "html.parser")
    # Extract text from the parsed HTML
    clean_text = soup.get_text(separator=" ")
    return str(clean_text)

# Example usage
# html_text = "<p>This is <b>bold</b> and <i>italic</i> text.</p>"
# clean_text = remove_html_tags(html_text)
# print(clean_text)

In [ ]:
def find_content(response, tag, id):
    soup = BeautifulSoup(response.content)

    content = soup.find_all(tag, id=id)
    if content:
        return (True, remove_html_tags(str(content)))
    else:
        return (False, '')


In [ ]:
import pandas as pd

df = pd.DataFrame()

# Find all anchor tags
anchor_tags = alpha_drug_name_tags

no_info_meds = 0
# Iterate over each anchor tag
for tag in anchor_tags:
    # Extract the link from the 'href' attribute
    link = webmd_base + tag.get('href')

    r = requests.get(link, headers=headers)

    # Check if the link exists
    if link:
        #Find the generic name of the drug if it exists
        soup = BeautifulSoup(r.content)
        alpha_gen = soup.find('h3', class_='drug-generic-name')
        alpha_generic_name = ''
        if(alpha_gen):
            alpha_generic_name = remove_html_tags(alpha_gen.get_text())
            alpha_generic_name.replace(' Generic Name(S): ', '')
        # Make a request using the link
        alpha_use = find_content(response=r, tag='div', id='uses-container')
        u_index = alpha_use[1].find('Uses')
        h2u_index = alpha_use[1].find('How to use')
        uses = alpha_use[1][u_index + len('Uses') : h2u_index]
        h2u = alpha_use[1][h2u_index + len('How to use') : ]
        h2u = h2u.replace(']', '').strip()
        uses = uses.strip()

        alpha_side = find_content(response=r, tag='div', id='sideeffects-container')
        side = alpha_side[1].replace('[', '').strip()[len('Side Effects') : ].replace(']', '').strip()

        alpha_warn = find_content(response=r, tag='div', id='warnings-container')
        warn = alpha_warn[1].replace('[', '').strip()[len('Warnings') : ].replace(']', '').strip()

        alpha_pre = find_content(response=r, tag='div', id='precautions-container')
        pre = alpha_pre[1].replace('[', '').strip()[len('Precautions') : ].replace(']', '').strip()
        if(warn):
            pre = warn + ' ' + pre

        alpha_inter = find_content(response=r, tag='div', id='interactions-container')
        inter = alpha_inter[1].replace('[', '').strip()[len('Interactions') : ].replace(']', '').strip()

        if(alpha_use[0] and alpha_side[0] and alpha_pre[0]):
            # print(f'Uses: {uses}')
            # print(f'How to use: {h2u}')
            # print(f'Side Effects: {side}')
            # print(f'Precautions: {pre}')
            # print(f'Interactions: {inter}')

            # Create a dictionary with column names as keys and data as values
            data = {
                'gen' : [alpha_generic_name],
                'drug': [tag.text.strip()],
                'uses': [uses],
                'h2u': [h2u],
                'precautions': [pre],
                'interactions': [inter]
            }

            new_df = pd.DataFrame(data)

            # Append new data to the existing DataFrame
            df = pd.concat([df, new_df], ignore_index=True)


        else:
            print(f'link not found for {tag.text.strip()}')
            no_info_meds += 1
print(f'{no_info_meds} have no information / out-going links')

link not found for a and d diaper rash cream
link not found for a thru z select women's tablet
link not found for a-carn 100 mg-a-cyst 37.5 mg-ala 25 mg-q10 mg-p.serine-broc capsule
link not found for aa 2%-d10w-low calcium-heparin solution, intravenous
link not found for aa 2.5%-d10w-calcium-heparin solution, intravenous
link not found for aa 3%-d10w-low calcium-heparin solution, intravenous
link not found for aa 3%-d5w-calcium-heparin solution, intravenous
link not found for aa 3.5%-d10w-calcium-heparin solution, intravenous
link not found for aa 4%-d10w-calcium-heparin solution, intravenous
link not found for aa-protein-selen-dex-biofl-blu capsule
link not found for b activ 400 mcg dfe capsule multivitamins
link not found for b and c topical ointment not applicable
link not found for b complex-c-e-fa-zn-lysine tablet
link not found for b complex-e-fa-min comb#35-soy tablet
link not found for b-12 lozenge
link not found for b-12 plus 5,000 mcg-100 mcg sublingual tablet
link not found

<ipython-input-5-003f7c94c39c>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


link not found for ga powder 15.1 g-500 kcal/100 g oral
link not found for gaba 100 mg-theanine mg-ashwagandha extract 225 mg capsule
link not found for gaba 250 mg-5htp 50 mg-mag mg-vit b6-folate 800 mcg dfe-b12 capsule
link not found for gaba 50 mg-5htp mg-theanine mg-taur 150 mg-multivit-min capsule
link not found for gaba soothe 100 mg-100 mg-225 mg capsule
link not found for gaba-5-htp-caffeine-arginine-choline-bromelain-herbal 1,400 mg capsule
link not found for gabadone
link not found for gabapentin (bulk) 100 % powder
link not found for gabapentin cream in metered-dose pump
link not found for galactose 400 kcal/100 gram oral powder
link not found for galaxtra 400 kcal/100 gram oral powder
link not found for galcanezumab-gnlm pen injector
link not found for gallium ga-68 dotatoc 18.5 mbq to 148 mbq/ml(0.5 4 mci/ml) iv soln radiopharmaceuticals
link not found for gallium ga-68 gozetotide 18.5 mbq to 185 mbq/ml (0.5-5 mci/ml) iv soln diagnostic radiopharmaceuticals - prostatic
lin

In [ ]:
df.head()

,gen,drug,uses,h2u,precautions,interactions
0,Generic Name(S): multivitamin-iron-folic acid,a thru z advanced formula,This medication is a multivitamin and iron p...,A Thru Z Advanced Formula Take this medicatio...,Accidental overdose of iron-containing produ...,See also How To Use section. Drug interactions...
1,Generic Name(S): a-c-e-zinc ox-selen aa-copper,a-c-e-zinc ox-selen aa-copper tablet,This medication is a multivitamin product us...,A-C-E-Zinc Ox-Selen AA-Copper Tablet Take this...,"If your brand of multivitamin contains iron, i...",Drug interactions may change how your medica...
2,Generic Name(S): vitamin B complex,b complex,This product is a combination of B vitamins ...,"B COMPLEX Take this medication by mouth , u...","Before taking this product, tell your doctor o...",Drug interactions may change how your medica...
3,Generic Name(S): vitamin b complex-folic acid,b complex 1 (with folic acid) 0.4 mg tablet mu...,This product is a combination of B vitamins ...,B Complex 1 (With Folic Acid) 0.4 Mg Tablet Mu...,"Before taking this product, tell your doctor o...",Drug interactions may change how your medica...
4,Generic Name(S): b3-b5-b6-b7-folic-b12-inosit-c,b complex-vitamin c 20 mg-5 mg-2 mg-75 mcg che...,This product is a combination of B vitamins ...,B Complex-Vitamin C 20 Mg-5 Mg-2 Mg-75 Mcg Che...,"Before taking this product, tell your doctor o...",Drug interactions may change how your medica...


In [ ]:
df['gen'] = df['gen'].str.replace('Generic Name(S):', '').str.strip()

In [ ]:
df.head()

,gen,drug,uses,h2u,precautions,interactions
0,multivitamin-iron-folic acid,a thru z advanced formula,This medication is a multivitamin and iron p...,A Thru Z Advanced Formula Take this medicatio...,Accidental overdose of iron-containing produ...,See also How To Use section. Drug interactions...
1,a-c-e-zinc ox-selen aa-copper,a-c-e-zinc ox-selen aa-copper tablet,This medication is a multivitamin product us...,A-C-E-Zinc Ox-Selen AA-Copper Tablet Take this...,"If your brand of multivitamin contains iron, i...",Drug interactions may change how your medica...
2,vitamin B complex,b complex,This product is a combination of B vitamins ...,"B COMPLEX Take this medication by mouth , u...","Before taking this product, tell your doctor o...",Drug interactions may change how your medica...
3,vitamin b complex-folic acid,b complex 1 (with folic acid) 0.4 mg tablet mu...,This product is a combination of B vitamins ...,B Complex 1 (With Folic Acid) 0.4 Mg Tablet Mu...,"Before taking this product, tell your doctor o...",Drug interactions may change how your medica...
4,b3-b5-b6-b7-folic-b12-inosit-c,b complex-vitamin c 20 mg-5 mg-2 mg-75 mcg che...,This product is a combination of B vitamins ...,B Complex-Vitamin C 20 Mg-5 Mg-2 Mg-75 Mcg Che...,"Before taking this product, tell your doctor o...",Drug interactions may change how your medica...


In [ ]:
df.to_csv('output.csv', index=False)
#df.to_csv('/content/gdrive/MyDrive/PMC_papers/output.csv', index=False)